# Midterm Project

## Imports

In [203]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text
from sklearn.ensemble import RandomForestRegressor

## Data Preparation

### Importing Dataset

In [204]:
df = pd.read_csv('full_dataset.csv')

In [205]:
df.dtypes

matchId         object
queueId          int64
gameVersion     object
teamId           int64
role            object
champion        object
gold_14          int64
xp_14            int64
kills_14         int64
deaths_14        int64
assists_14       int64
plates_14        int64
towers_14        int64
dragons_14       int64
heralds_14       int64
grubs_14         int64
gold_20          int64
xp_20            int64
damage_20      float64
win               bool
gold_growth      int64
xp_growth        int64
kda_14         float64
dtype: object

### Preliminary Cleaning

In [206]:
# Standardize columns and string data
df.columns = df.columns.str.lower()

# Standardize string values
string_columns = list(df.dtypes[df.dtypes == 'object'].index)
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')
    
# Handle missing numeric values
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(0)

In [207]:
df

,matchid,queueid,gameversion,teamid,role,champion,gold_14,xp_14,kills_14,deaths_14,assists_14,plates_14,towers_14,dragons_14,heralds_14,grubs_14,gold_20,xp_20,damage_20,win,gold_growth,xp_growth,kda_14
0,euw1_7391567515,420,15.9.678.1456,100,top,kennen,5758,6739,5,2,3,4,0,1,0,0,9252,10317,168.26,False,3494,3578,2.666667
1,euw1_7391567515,420,15.9.678.1456,100,jungle,gwen,4528,4933,2,3,1,4,0,1,0,0,8285,9120,111.26,False,3757,4187,0.750000
2,euw1_7391567515,420,15.9.678.1456,100,mid,velkoz,4667,6811,1,2,1,4,0,1,0,0,7138,10182,143.76,False,2471,3371,0.666667
3,euw1_7391567515,420,15.9.678.1456,100,adc,draven,5348,5531,2,2,0,4,0,1,0,0,9767,8954,113.86,False,4419,3423,0.666667
4,euw1_7391567515,420,15.9.678.1456,100,support,leona,3306,2599,1,3,2,4,0,1,0,0,5342,5373,3810.00,False,2036,2774,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126725,euw1_7414672500,420,15.10.680.4378,200,top,rumble,5304,7072,5,2,0,8,1,0,0,0,7008,9470,144.22,True,1704,2398,1.666667
126726,euw1_7414672500,420,15.10.680.4378,200,jungle,pantheon,5176,5073,3,3,2,8,1,0,0,0,8758,9092,9393.00,True,3582,4019,1.250000
126727,euw1_7414672500,420,15.10.680.4378,200,mid,yasuo,4699,6378,1,2,2,8,1,0,0,0,7592,10744,4788.00,True,2893,4366,1.000000
126728,euw1_7414672500,420,15.10.680.4378,200,adc,jhin,6074,4762,6,1,3,8,1,0,0,0,9666,8504,149.53,True,3592,3742,4.500000


In [208]:
# Check which numeric columns are all zeros and could be safely removed
all_zero_cols = [col for col in numeric_cols if (df[col] == 0).all()]

print("Numeric columns with all zero values:")
print(all_zero_cols)

Numeric columns with all zero values:
['heralds_14', 'grubs_14']


### Feature Engineering

In [209]:
# Separate teams
blue_team = df[df['teamid'] == 100].copy()
red_team  = df[df['teamid'] == 200].copy()

In [210]:
# Aggregate player stats per team (keep per-role stats)
roles = ['top', 'jungle', 'mid', 'adc', 'support']
stats_14 = ['gold_14', 'xp_14', 'kda_14']
stats_20 = ['gold_20', 'xp_20', 'damage_20']
stats = stats_14 + stats_20
objectives = ['dragons_14', 'towers_14', 'plates_14']

In [211]:
def pivot_team(team_df, prefix):
    # Start with unique match IDs
    team_pivot = pd.DataFrame({'matchid': team_df['matchid'].unique()})
    
    for role in roles:
        role_df = team_df[team_df['role'] == role].copy()
        role_df = role_df[['matchid', 'champion'] + stats]  # keep relevant columns
        
        # Rename columns with role + prefix
        rename_dict = {'champion': f'champion_{role}_{prefix}'}
        rename_dict.update({stat: f'{stat}_{role}_{prefix}' for stat in stats})
        role_df = role_df.rename(columns=rename_dict)
        
        # Merge into pivot table on matchid
        team_pivot = team_pivot.merge(role_df, on='matchid', how='left')
    
    # Add team objectives (take first value, not sum)
    for obj in objectives:
        obj_df = team_df.groupby('matchid')[obj].first().reset_index()
        obj_df = obj_df.rename(columns={obj: f'{obj}_{prefix}'})
        team_pivot = team_pivot.merge(obj_df, on='matchid', how='left')
    
    return team_pivot

blue_team_pivot = pivot_team(blue_team, 'blue')
red_team_pivot  = pivot_team(red_team, 'red')

In [212]:
# Merge teams into match-level dataframe
match_df = pd.merge(
    blue_team_pivot,
    red_team_pivot,
    on='matchid',
    how='inner'
)

In [213]:
# Target variable
match_df['target_win'] = (blue_team.groupby('matchid')['win'].first().astype(int)).values

In [214]:
# Reorder columns: blue team, then red team, then target
def build_team_cols(prefix):
    cols = []
    for role in roles:
        cols.append(f'champion_{role}_{prefix}')
        for stat in stats:
            cols.append(f'{stat}_{role}_{prefix}')
    for obj in objectives:
        cols.append(f'{obj}_{prefix}')
    return cols

blue_cols = build_team_cols('blue')
red_cols = build_team_cols('red')

ordered_cols = ['matchid'] + blue_cols + red_cols + ['target_win']
ordered_cols = [c for c in ordered_cols if c in match_df.columns]

match_df = match_df[ordered_cols]

In [218]:
match_df

,matchid,champion_top_blue,gold_14_top_blue,xp_14_top_blue,kda_14_top_blue,gold_20_top_blue,xp_20_top_blue,damage_20_top_blue,champion_jungle_blue,gold_14_jungle_blue,xp_14_jungle_blue,kda_14_jungle_blue,gold_20_jungle_blue,xp_20_jungle_blue,damage_20_jungle_blue,champion_mid_blue,gold_14_mid_blue,xp_14_mid_blue,kda_14_mid_blue,gold_20_mid_blue,xp_20_mid_blue,damage_20_mid_blue,champion_adc_blue,gold_14_adc_blue,xp_14_adc_blue,kda_14_adc_blue,gold_20_adc_blue,xp_20_adc_blue,damage_20_adc_blue,champion_support_blue,gold_14_support_blue,xp_14_support_blue,kda_14_support_blue,gold_20_support_blue,xp_20_support_blue,damage_20_support_blue,dragons_14_blue,towers_14_blue,plates_14_blue,champion_top_red,gold_14_top_red,xp_14_top_red,kda_14_top_red,gold_20_top_red,xp_20_top_red,damage_20_top_red,champion_jungle_red,gold_14_jungle_red,xp_14_jungle_red,kda_14_jungle_red,gold_20_jungle_red,xp_20_jungle_red,damage_20_jungle_red,champion_mid_red,gold_14_mid_red,xp_14_mid_red,kda_14_mid_red,gold_20_mid_red,xp_20_mid_red,damage_20_mid_red,champion_adc_red,gold_14_adc_red,xp_14_adc_red,kda_14_adc_red,gold_20_adc_red,xp_20_adc_red,damage_20_adc_red,champion_support_red,gold_14_support_red,xp_14_support_red,kda_14_support_red,gold_20_support_red,xp_20_support_red,damage_20_support_red,dragons_14_red,towers_14_red,plates_14_red,target_win
0,euw1_7391567515,kennen,5758,6739,2.666667,9252,10317,168.26,gwen,4528,4933,0.750000,8285,9120,111.26,velkoz,4667,6811,0.666667,7138,10182,143.76,draven,5348,5531,0.666667,9767,8954,113.86,leona,3306,2599,0.750000,5342,5373,3810.00,1,0,4,irelia,5110,5880,0.500000,8587,10024,132.12,pantheon,6405,5808,2.500000,9780,9625,156.80,ekko,5058,6774,1.500000,8984,10862,107.28,aphelios,3852,4427,0.666667,6544,7625,5789.00,sona,4542,3774,7.000000,6599,7129,7887.00,0,0,4,1
1,euw1_7401794131,irelia,4402,5341,0.400000,6961,8323,128.38,vi,5407,5955,4.000000,8903,10545,125.67,zoe,4891,7055,3.000000,8778,10655,202.54,missfortune,5975,5474,3.000000,10092,9051,145.41,neeko,4819,3841,12.000000,7330,5796,135.95,2,0,6,poppy,5870,6130,1.333333,8847,8389,164.91,viego,4853,4699,1.250000,7148,7749,9151.00,galio,3908,5624,0.200000,5838,8845,9513.00,xayah,4812,4735,0.333333,6755,6978,123.37,janna,3447,3068,0.750000,5832,5025,5662.00,0,1,7,0
2,euw1_7413358722,chogath,4556,6797,1.000000,6667,9889,140.12,monkeyking,5233,5626,1.500000,7839,9444,5544.00,anivia,4510,5664,1.333333,7320,9275,119.55,ashe,4545,4973,1.000000,7723,8004,167.65,senna,3424,3229,0.800000,6131,6527,130.80,1,0,4,drmundo,4804,6586,0.000000,10064,11257,135.60,ivern,4097,4170,1.333333,6594,8398,8484.00,sion,5710,6201,1.250000,8867,10649,169.40,sivir,4250,3824,2.500000,6072,6531,9270.00,soraka,4121,3150,3.500000,5736,5355,7285.00,0,0,7,1
3,euw1_7380973262,jayce,4441,5499,0.833333,5329,6755,7268.00,nidalee,5325,5562,2.000000,6632,7728,4743.00,ahri,4518,6652,1.000000,5585,8240,7303.00,missfortune,5078,5094,1.500000,6391,6464,7268.00,alistar,3339,2890,0.800000,3898,3542,3460.00,0,0,3,taliyah,5772,7045,1.000000,7631,8931,115.19,talon,5258,5791,4.000000,7129,8407,5270.00,katarina,5301,7351,2.000000,6999,9481,6561.00,varus,4949,5595,2.000000,6653,7391,9651.00,rell,3158,2825,0.666667,3953,3704,3289.00,2,0,6,0
4,euw1_7389215482,shen,3831,5583,1.333333,6274,9636,5475.00,graves,5814,5589,2.333333,10440,10389,132.23,ahri,4896,5792,0.800000,7619,9331,102.50,corki,7068,5708,4.000000,9723,8865,137.90,gragas,4481,3276,3.666667,6457,5412,6920.00,1,1,7,heimerdinger,5847,7371,2.000000,8785,11190,160.34,briar,4409,4682,0.571429,6636,7193,7954.00,taliyah,6092,5957,2.333333,8765,9204,143.88,ekko,4235,4541,0.500000,7186,7751,6394.00,nautilus,4783,2640,1.750000,6373,5312,5894.00,0,1,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12